In [24]:
from transformers import AutoTokenizer, AutoModelForCausalLM

# 本地模型保存路径
model_dir = "C:/Users/tix/.cache/modelscope/hub/damo/nlp_polylm_assistant_13b_text_generation"

# 加载 tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_dir)

# 加载模型
model = AutoModelForCausalLM.from_pretrained(model_dir, local_files_only=True)

# 在离线模式下使用模型和 tokenizer
input_ids = tokenizer.encode("你好", return_tensors="pt")
output = model.generate(input_ids, max_length=50, do_sample=True)
generated_text = tokenizer.decode(output[0], skip_special_tokens=True)

print(generated_text)

ValueError: Couldn't instantiate the backend tokenizer from one of: 
(1) a `tokenizers` library serialization file, 
(2) a slow tokenizer instance to convert or 
(3) an equivalent slow tokenizer class to instantiate and convert. 
You need to have sentencepiece installed to convert a slow tokenizer to a fast one.

In [9]:
# 定义数据集
class Dataset(torch.utils.data.Dataset):
    def __init__(self, split):
        mode_name_or_path = r'C:\Users\tix\.cache\modelscope\hub\datasets\damo\WMT-Chinese-to-English-Machine-Translation-Medical'
        self.dataset = load_from_disk(mode_name_or_path)[split]

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, i):
        text = self.dataset[i]['text']
        label = self.dataset[i]['label']
        return text, label
#处理一下方便后续的数据处理
import sqlite3
from transformers import AutoTokenizer

class Dataset(torch.utils.data.Dataset):
    def __init__(self, split):
        self.connection = sqlite3.connect(r'C:\Users\tix\.cache\modelscope\hub\datasets\damo\WMT-Chinese-to-English-Machine-Translation-Medical')
        self.cursor = self.connection.cursor()
        self.cursor.execute(f"SELECT text, label FROM your_table WHERE split='{split}'")
        self.data = self.cursor.fetchall()
        self.tokenizer = AutoTokenizer.from_pretrained("gpt2")

    def __len__(self):
        return len(self.data)

    def __getitem__(self, i):
        text, label = self.data[i]
        encoded_input = self.tokenizer.encode_plus(
            text,
            padding='max_length',
            truncation=True,
            max_length=512,
            return_tensors='pt'
        )
        input_ids = encoded_input['input_ids'].squeeze()
        attention_mask = encoded_input['attention_mask'].squeeze()
        return input_ids, attention_mask, labe


In [ ]:
from transformers import AutoTokenizer
from collections import defaultdict

# 使用 GPT-2 tokenizer 将输入分解为单词:
tokenizer = AutoTokenizer.from_pretrained("gpt2")
word_freqs = defaultdict(int)
for text in corpus:
words_with_offsets = tokenizer.backend_tokenizer.pre_tokenizer.pre_tokenize_str(text) new_words = 
[word for word, offset in words_with_offsets]
for word in new_words: word_freqs[word] += 1
# 计算基础词典, 这里使用语料库中的所有字符:
alphabet = []
for word in word_freqs.keys():
for letter in word:
if letter not in alphabet: alphabet.append(letter)
alphabet.sort()

# 增加特殊 Token 在字典的开头，GPT-2 中仅有一个特殊 Token``<|endoftext|>''表示文本结束
vocab = ["<|endoftext|>"] + alphabet.copy()

# 将单词切分为字符
splits = {word: [c for c in word] for word in word_freqs.keys()}

#compute_pair_freqs 函数用于计算字典中所有词元对的频率
def compute_pair_freqs(splits): pair_freqs = defaultdict(int)
     for word, freq in word_freqs.items(): 
            split = splits[word]
            if len(split) == 1:
                continue
            for i in range(len(split) - 1): 
                pair = (split[i], split[i + 1]) 
                pair_freqs[pair] += freq
                return pair_freqs




In [ ]:
import torch
from torch.utils.data import DataLoader
from transformers import AutoTokenizer, AutoModel, AdamW, get_scheduler
from datasets import load_from_disk
import os

# 定义数据加载器
class Dataset(torch.utils.data.Dataset):
    def __init__(self, split):
        mode_name_or_path = r'C:\Users\tix\.cache\modelscope\hub\datasets\damo\WMT-Chinese-to-English-Machine-Translation-Medical'
        self.dataset = load_from_disk(mode_name_or_path)[split]

    def __getitem__(self, idx):
        data = self.dataset[idx]
        input_text = data['chinese']
        target_text = data['english']

        # 使用 tokenizer 对文本进行处理
        input_encoding = tokenizer(input_text, padding='max_length', truncation=True, max_length=128)
        target_encoding = tokenizer(target_text, padding='max_length', truncation=True, max_length=128)

        # 将编码后的数据转换为 PyTorch 张量
        input_ids = torch.tensor(input_encoding['input_ids'])
        attention_mask = torch.tensor(input_encoding['attention_mask'])
        token_type_ids = torch.tensor(input_encoding['token_type_ids'])
        target_ids = torch.tensor(target_encoding['input_ids'])

        return input_ids, attention_mask, token_type_ids, target_ids

    def __len__(self):
        return len(self.dataset)


# 加载预训练模型和 tokenizer
model_path = 'damo/nlp_polylm_assistant_13b_text_generation'
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModel.from_pretrained(model_path)

# 定义超参数
batch_size=8
epochs=10

# 准备数据
train_dataset = Dataset('train')
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

# 定义优化器
optimizer = AdamW(model.parameters(), lr=5e-4)

# 定义1oss函数
criterion = torch.nn.CrossEntropyLoss(ignore_index=tokenizer.pad_token_id)

# 定义学习率调节器
scheduler = get_scheduler(name='linear', num_warmup_steps=0, num_training_steps=len(train_loader)*epochs, optimizer=optimizer)

# 将模型切换到训练模式
model.train()

# 定义保存模型的路径
model_dir = r'C:\Users\tix\.cache\modelscope\hub\workshop\model'

# 训练模型
for epoch in range(epochs):
    for i, (input_ids, attention_mask, token_type_ids, target_ids) in enumerate(train_loader):
        # 梯度清零
        optimizer.zero_grad()

        # 前向传播
        outputs = model(input_ids=input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids, decoder_input_ids=target_ids[:, :-1])[0]

        # 计算loss
        loss = criterion(outputs.view(-1, outputs.shape[-1]), target_ids[:, 1:].contiguous().view(-1))

        # 反向传播
        loss.backward()

        # 更新参数
        optimizer.step()

        # 调整学习率
        scheduler.step()

        # 输出loss
        if i % 10 == 0:
            print('Epoch:', epoch, 'Iteration:', i, 'Loss:', loss.item())

    # 每个 epoch 保存一次模型
    torch.save(model.state_dict(), os.path.join(model_dir, f'epoch{epoch}.pt')) 

In [ ]:
# 用训练好的模型 做一些简单的翻译
import sys
from PyQt5.QtWidgets import *
from transformers import pipeline,AutoTokenizer, AutoModelForSeq2SeqLM

tokenizer = AutoTokenizer.from_pretrained("damo/nlp_polylm_assistant_13b_text_generation")
model = AutoModelForSeq2SeqLM.from_pretrained("damo/nlp_polylm_assistant_13b_text_generation")
translation = pipeline('translation_zh_to_en',model=model,tokenizer=tokenizer)
# print('加载完成')


def translationPytorch(word):
  res = translation(word)[0]['translation_text']
  return  res


def isnumber(string):
  a = string[:1]
  if a == "0" or a == "1" or a == "2" or a == "3" or a == "4" or a == "5" or a == "6" or a == "7" or a == "8" or a == "9":
    return True
  else:
    return False

def write_txt(path, content):
  '''实现TXT文档的写方法'''
  with open(path, 'a+',encoding='utf-8') as f:
    f.write(content + "\n")


def transtxt(pathfrom,pathto):
  data = []
  file = open(pathfrom, 'r',encoding='utf-8')
  file_data = file.readlines()
  for row in file_data:
    tmp_list = row.split('\n')
    data.append(tmp_list)
  for indexi in data:
    for words in indexi:
      if len(words) > 0:
        if isnumber(words):
          # print(words)
          # write
          write_txt(pathto, words)

        else:
          tran = translationPytorch(words)
          # print(words)
          # print(tran)
          # write tran
          write_txt(pathto, words)
          write_txt(pathto, tran)


class example(QWidget):
    position=''
    def __init__(self):
        super(example, self).__init__()
        # 窗口标题
        self.setWindowTitle('请拖拽进需要执行的文件')
        # 定义窗口大小
        self.resize(500, 400)
        self.QLabl = QLabel(self)
        self.QLabl.setGeometry(2, 200, 4000, 90)
        # 调用Drops方法
        self.setAcceptDrops(True)

    def dragEnterEvent(self, evn):
        self.setWindowTitle('翻译中')
        self.QLabl.setText('已完成：\n' + evn.mimeData().text()+'的翻译\n结果在：D:/result.txt')
        self.position = evn.mimeData().text()
        pathfrom = self.position
        pathfrom = pathfrom[8:]
        pathto = "D:/result.txt"
        transtxt(pathfrom,pathto)
        evn.accept()
if __name__ == "__main__":
    app = QApplication(sys.argv)
    e = example()
    e.show()
    sys.exit(app.exec_())
